Installing the Everest agent:

In [ ]:
!~/bin/everest.sh

Overwriting the configuration file from backup:

In [ ]:
!cp ~/everest_agent/everest_agent/agent.conf.default ~/everest_agent/conf/agent.conf; echo -n AGENTTOKEN > agent-token.txt

Writing a new Everest agent token to the configuration file, saving resource ID:

In [ ]:
#include <cstdlib>
#include <iostream>
#include <fstream>
#include <string>
using namespace std;
{
    string old_token = "AGENTTOKEN";
    {ifstream file("agent-token.txt"); if(file) file >> old_token;} 
    cout << "Please enter the Everest agent Token:" << endl;
    string new_token; cin >> new_token;
    ofstream file("agent-token.txt"); file << new_token;
    string sed_command = "sed -i -e 's="+old_token+"="+new_token+"=' ~/everest_agent/conf/agent.conf";
    system(sed_command.c_str());
    cout << "Please enter the Everest agent ID:" << endl;
    string agent_ID; cin >> agent_ID;
    ofstream idfile("agent-ID.txt"); idfile << agent_ID;
 }

Starting the Everest agent:

In [ ]:
#include <cstdlib>
#include <fstream>
#include <iostream>
using namespace std;
{ifstream file("../../everest_agent/agent.pid"); if(file){cout << "Everest agent is already running, please stop it first.";return;}}
system("~/everest_agent/bin/start.sh -c ~/everest_agent/conf/agent.conf");
{ifstream file("../../everest_agent/agent.pid"); if(file){cout << "Everest agent is running.";}}

Opening a session, getting a session token:

In [ ]:
#pragma cling load("libcurl")
#pragma cling add_include_path("../../lib/opt")
#include "../../lib/everest.hpp"
#include <fstream>
#include "xcpp/xdisplay.hpp"
using namespace std;
{
    {ifstream file("session-token.txt"); if(file) {cout << "The session is already open. Please close it first."; return;}}
    string login,pass,token;
    cout << "Please enter your Everest login:"    << endl; cin >> login;
    cout << "Please enter your Everest password:" << endl; cin >> pass;
    xcpp::clear_output(true);
    templet::everest_engine teng(login.c_str(),pass.c_str());
    if(!teng){ std::cout << "There was an error logging into Everest." << endl; return; }
    teng.save_access_token();
    teng.get_access_token(token);
    ofstream file("session-token.txt"); file << token;
    cout << "Your session token is: " << token;
}

Running a test application on the Everest agent:

In [ ]:
#pragma cling load("libcurl")
#pragma cling add_include_path("../../lib/opt")
#include "../../lib/everest.hpp"
#include <fstream>
using namespace std;
{
    ifstream file("session-token.txt"),idfile("agent-ID.txt"); 
    if(!file){ cout << "The session is not open. Please open it first."; return;}
    if(!idfile){cout << "The Everest agent ID is absent. Please setup the agent first.";return;}
    string token, agentID; file >> token; idfile >> agentID;
    templet::everest_engine teng(token.c_str());
    if(!teng){ cout << "There was an error connecting to Everest." << endl; return; }
    const char* sleep_app_id = "52b1ce8f3d00007a007a7189";// See 'Getting the ID and description...'
    templet::everest_task sleep_task;
    sleep_task.app_id(sleep_app_id);
    sleep_task.engine(teng);
    json in;
    in["name"] = string("Sleep");
    in["inputs"]["duration"]=1;
    in["resources"][0]=agentID;
    if(sleep_task.submit(in)) cout << "Task submitted successfully." << endl;
    teng.run();
    templet::everest_error cntxt;
    if (teng.error(&cntxt)) {
        cout << "Task engine error:" << endl; templet::everest_engine::print_error(&cntxt);
        return;
    }
    cout << "Test task completed successfully.";
}

Getting the ID and description of arbitrary Everest applications by name:

In [ ]:
#pragma cling load("libcurl")
#pragma cling add_include_path("../../lib/opt")
#include "../../lib/everest.hpp"
#include <fstream>
using namespace std;
{
    ifstream file("session-token.txt"); 
    if(!file){ cout << "The session is not open. Please open it first."; return;}
    string token; file >> token;
    templet::everest_engine teng(token.c_str());
    if(!teng){ cout << "There was an error connecting to Everest." << endl; return; }
    cout << "Please enter the name of the Everest application (or part of the name):";
    string app_name; cin >> app_name;
    cout << "These are the descriptions of Everest apps named '" << app_name <<"':" << endl << endl;
    teng.print_app_description(app_name.c_str());
}

Closing the Everest session:

In [ ]:
#pragma cling load("libcurl")
#pragma cling add_include_path("../../lib/opt")
#include "../../lib/everest.hpp"
#include <fstream>
#include <cstdio>
using namespace std;
{
    ifstream file("session-token.txt"); 
    if(!file){ cout << "The session is not open. Please open it first."; return;}
    string token; file >> token;
    templet::everest_engine teng(token.c_str());
    if(!teng){ cout << "There was an error connecting to Everest." << endl; return; }
    teng.save_access_token(false);
    remove("session-token.txt");
    cout << "The session is closed." << endl;
}

Stopping the Everest agent:

In [ ]:
!kill $(cat ~/everest_agent/agent.pid); rm ~/everest_agent/agent.pid